In [10]:
#Script to open netcdf and plot
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cartopy #Instead of from mpl_toolkits.basemap import Basemap "conda install -c conda-forge cartopy"
from netCDF4 import Dataset #"conda install -c anaconda netcdf4"
#from scipy.io import netcdf -- Alternative to netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#hourly data
path = '/Users/gjmacdonald/Documents/Sea Ice Project/AutoSeaIceProduc2/adaptor.mars.internal-1618949220.213186-8983-19-61ed83d6-dfa1-4203-8537-42b048d2ab5d.nc'#Path to file
#outpath = '/Users/gjmacdonald/Documents/Sea Ice Project/DailyWind/DailyWindJPGs/'

#Wind speed sqrt(u^2 + v^2)

#This works for animation so far!
from mpl_toolkits.axes_grid1 import make_axes_locatable

data = Dataset(path)
data.variables

OrderedDict([('longitude',
              <class 'netCDF4._netCDF4.Variable'>
              float32 longitude(longitude)
                  units: degrees_east
                  long_name: longitude
              unlimited dimensions: 
              current shape = (85,)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('latitude',
              <class 'netCDF4._netCDF4.Variable'>
              float32 latitude(latitude)
                  units: degrees_north
                  long_name: latitude
              unlimited dimensions: 
              current shape = (19,)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('time',
              <class 'netCDF4._netCDF4.Variable'>
              int32 time(time)
                  units: hours since 1900-01-01 00:00:00.0
                  long_name: time
                  calendar: gregorian
              unlimited dimensions: 
              current shape = (720,)
 

In [11]:
import xarray as xr                                                    # import xarray library. Necessary for daily

##GET LATS, LONS AND TIME
lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
#wind = data.variables['si10'][:,:,:] #wind
#u = data.variables['u10'][:,:,:]
#v = data.variables['v10'][:,:,:]

##SET UP TO CONVERT HOURLY TO DAILY
ds_nc = xr.open_dataset(path)                        # read the file for daily
tp_daily = ds_nc.resample(time='D').mean(dim='time') # dimensions (time: 2, latitude: 721, longitude: 1440)

##GET DAILY WIND SPEED -- same as other wind stuff
u = tp_daily.variables['u10'][:,:,:]
v = tp_daily.variables['v10'][:,:,:]
u = u.values #Converts from pandas datafrome to numpy array. Because hourly to daily method makes df
v = v.values #now recommend df.to_numpy() 
windspeed = np.sqrt(u**2 + v**2) #Wind speed sqrt(u^2 + v^2)

In [12]:
##Extract data
#data.variables.keys() - check data in netcdf
DewPoint2m = tp_daily.variables['d2m'][:,:,:]
DewPoint2m = DewPoint2m.values

Temp2m = tp_daily.variables['t2m'][:,:,:]
Temp2m = Temp2m.values

SurfPres = tp_daily.variables['sp'][:,:,:]
SurfPres = SurfPres.values

InSol = tp_daily.variables['ssrd'][:,:,:] #'ssrd' = 'surface solar radiation downwards'
InSol = InSol.values

InTherm = tp_daily.variables['strd'][:,:,:]
InTherm = InTherm.values

In [13]:
## Calculate Heat Flux Terms
#Net heat flux = Q = (1 − α)Si + Li − Lo + Fs + Fe (from Cheng et al 2017)
#Si = incoming solar, Li incoming LW
#Need to calculate: Lo (outgoing LW)
#Need to calculate: Fs (sensinble heat flux)
#Need to calculate Fe (latent heat flux)

#This seems to be constant. Check salinity in my region
#OUTGOING LW = εσT04
#ε = LW emissivity of open water= 0.99 
#σ= Stefan Boltzmann constant (=5.67 × 10−8 W·m−2·K−4)
#T0 = 271.27 (maybe later adjust to Amundsen salinity, but seems simmilar)
stef = 5.67*(10**-8)
T0 = 271.27 #This could change if salinity dif in Amundsen
OutTherm = 0.99 * stef * (T0**4)

##Sensible heat flux
# Fs = ρacpCsU(Ta − T0)
# 1.3 kg·m−3 * 1004 J·kg−1·K−1 * 0.00144 * windspeed *(Temp2m - T0fromabove)
# T0 = Ts = 273.15 − 0.0137 − 0.05199sw − 0.00007225sw2 = 271.27
#Temp2m is in K
#Is 
SensHeat = 1.3 * 1004 * 0.00144 * windspeed * (Temp2m - T0)

#---------------------------------------
##LATENT HEAT FLUX
#Fe = 0.622ρaLvCeU(rea − es)/P0

#0.622 * 1.3 kg·m−3 *Lv * 0.00144 * windspeed (actualVP - saturationVP) / SurfPres

#Lv = [2.501 − 0.00237(Ts − 273.15)] × 106 = 2.51 × 10 J·kg  **Check isnt dif for here

#actualVP = rea =611.21×109.8094(Td−273.15)/(Td+0.71):
#6.11.21 *10^(9.8094(DewPoint2m - 273.15)/Depoint2m+0.71)

#saturationVP = 611.21 * 10^(90.8094(T0 -273.15)/(T0+0.71))

#Need LV, rea, es. LV is dependent on T0! (which is dependent on salinity)
Lv = 2.51 * (10**6) # Lv = [2.501 − 0.00237(Ts − 273.15)] × 106 -- would change if use different salinity to get Ts/T0
rea = 611.21 * 10**(9.8094*(DewPoint2m - 273.15)/(DewPoint2m + 0.71))
es  = 611.21 * 10**(9.8094*(T0         - 273.15)/(T0         + 0.71))
LatHeat = (0.622 * 1.3 * Lv * 0.00144 * windspeed * (rea - es))/SurfPres

##WHAT DO I NEED TO DO.
#DETERMINE WHAT T0 IS. IS IT THE SAME OR DIFFERENT? CHECK SALT TO SEE IF CHANGE.
#THE REST IS PLUGGING IN TO GET HEAT FLUX. NUT NEED SIC FOR FINAL PRODUCTION.

# Perhaps consider saving netcdf tiles, or final heatflux output, as geotiff/raster
#(Saving calculated heatflux as raster seems more sensible if do this as fewer files - just one per day)
#But perhaps want to do reprojection on whole netcdf at start (and resample?). Still, can leave til after calcs coded.

In [14]:
##CALCULATE NET HEAT FLUX USING ALL CLIMATE VARIABLES
#Q = (1 − α)Si + Li − Lo + Fs + Fe (1)
#α= albedo of open water = 0.06 
NetHeat = (1-0.06)*InSol + InTherm - OutTherm + SensHeat + LatHeat


In [15]:
NetHeat.shape

(30, 19, 85)

In [16]:
## Write to netcdf
#https://unidata.github.io/python-training/workshop/Bonus/netcdf-writing/
import netCDF4 as nc
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass
ncfile = Dataset('/Users/gjmacdonald/Documents/Sea Ice Project/AutoSeaIceProduc2/HeatFluxOct_B.nc',mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)


lat_dim = ncfile.createDimension('latitude', 19)     # latitude axis. Check shape NetHeat.shape - day/lat/lon
lon_dim = ncfile.createDimension('longitude', 85)    # longitude axis
time_dim = ncfile.createDimension('DAY', None)

#timeX = ds.createDimension('DAY', None)
#latX = ds.createDimension('latitutde', 13)
#lonX = ds.createDimension('longitude', 85)


#valueX = ds.createVariable('NetHeat', 'f4', ('time', 'lat', 'lon',))

ncfile.title='My heatflux data, Oct 2020 daily'

latitude = ncfile.createVariable('latitude', np.float32, ('latitude',))
longitude = ncfile.createVariable('longitude', np.float32, ('longitude',))
DAR = ncfile.createVariable('DAY', np.float64, ('DAY',))
temp = ncfile.createVariable('temp',np.float64,('DAY','latitude','longitude'))

print(temp)


#timesVar = ds.createVariable('DAY', 'f4', ('DAY',))
#latsVar = ds.createVariable('latitude', 'f4', ('latitude',))#others does diff for middle var
#lonsVar = ds.createVariable('longitude', 'f4', ('longitude',))

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 

<class 'netCDF4._netCDF4.Variable'>
float64 temp(DAY, latitude, longitude)
unlimited dimensions: DAY
current shape = (0, 19, 85)
filling on, default _FillValue of 9.969209968386869e+36 used



/Users/gjmacdonald/opt/anaconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


In [17]:
latitude[:] = lats
longitude[:] = lons
temp[:,:,:] = NetHeat


In [18]:
print(ncfile)
# close the Dataset.
ncfile.close(); print('Dataset is closed!')
#As an nc file, for some reason only opens on QGIS on my old mac. NEXT STEPS:

#Convert to geotiff: gdal_translate -of GTiff HeatFluxOct.nc HeatFluxTif.tiff

# gdal_translate -a_srs EPSG:4326 HeatFluxTif2.tiff HeatFluxTif3.tiff -- add projection info. Doesn't work on new mac for some reason

# gdalwarp -t_srs EPSG:3976 HeatFluxTif3.tiff HeatFluxTif_3976.tiff -- reproject to NSIDC

# gdalwarp -tr 3125 -3125 HeatFluxTif_3976.tiff HeatFluxRes.tiff -- change resolution to match SIC (3.125km)

#gdalwarp -te -1810937.500 -1445312.500 -1064062.500 -554687.500 HeatFluxRes.tiff HFNEW_SICcords.tiff -- align pixels bween rasters
#^ xmin xmax ymin ymax of the sea ice concentration

#CONDENSED and better:
#gdal_translate -of GTiff -a_srs EPSG:4326 HeatFluxOct_B.nc HeatFluxTif_cond.tiff
#gdalwarp -t_srs EPSG:3976 -tr 3125 -3125 -te -1810937.500 -1445312.500 -1064062.500 -554687.500 HeatFluxTif_cond.tiff HeatFluxTif_cond2.tiff 

#Clipped in QGIS. Clip by extent.
#Make sure Match extent of clipped raster to extent of mask layer is selected. This was problem with code below

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: My heatflux data, Oct 2020 daily
    dimensions(sizes): latitude(19), longitude(85), DAY(30)
    variables(dimensions): float32 latitude(latitude), float32 longitude(longitude), float64 DAY(DAY), float64 temp(DAY,latitude,longitude)
    groups: 

Dataset is closed!


/Users/gjmacdonald/opt/anaconda3/envs/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  """Entry point for launching an IPython kernel.


In [21]:
##THIS BELOW CLIPS BUT NOT TO RIGHT EXTENT, DONT USE, USE QGIS/GDAL, SEE ABOVE

##Clip to SIC mask #As is clips twice, so makes two files. See if can change script, or change destination file
import os, fnmatch
#from subprocess import call
from osgeo import gdal
inFolder = '/Users/gjmacdonald/Documents/Sea Ice Project/AutoSeaIceProduc2/'
outFolder = '/Users/gjmacdonald/Documents/Sea Ice Project/AutoSeaIceProduc2/Masked/'
mask = "/Users/gjmacdonald/Documents/Sea Ice Project/SIC_AMSR2/Mask/Mask3.shp" #Mask3 made from Nov 2020
#warp= gdal.Warp(outRaster,raster, cutlineDSName = mask, dstNodata = 255)

os.chdir (inFolder)

def findRasters (path, filter):
    for root, dirs, files in os.walk(path, filter):
        for file in fnmatch.filter(files, filter):
            yield os.path.join (root, file)
            

for raster in findRasters (inFolder, '*.tiff'): #Find all tifs
    (infilepath, infilename)= os.path.split (raster)
    print(infilename)
    outRaster= outFolder+ 'clip_'+ infilename
    print(outRaster)
    warp= gdal.Warp(outRaster,raster, cutlineDSName = mask, dstNodata = 255)

HeatFluxTifProj_3976_3125km_Align.tiff
/Users/gjmacdonald/Documents/Sea Ice Project/AutoSeaIceProduc2/Masked/clip_HeatFluxTifProj_3976_3125km_Align.tiff
clip_HeatFluxTifProj_3976_3125km_Align.tiff
/Users/gjmacdonald/Documents/Sea Ice Project/AutoSeaIceProduc2/Masked/clip_clip_HeatFluxTifProj_3976_3125km_Align.tiff
